In [1]:
# Simple CNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.models import Model
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.layers import Input
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import LSTM
from keras.layers import Embedding

In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# this is the size of our encoded representations
encoding_dim = 128  # 32 floats -> compression of factor 24.5, assuming the input is 3074 floats

In [4]:
# this is our input placeholder
input_img = Input(shape=(3072,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(3072, activation='sigmoid')(encoded)
# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
# this model maps an input to its encoded representation
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [6]:
# normalize inputs from 0-255 to 0.0-1.0
test_data = X_train
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))

In [7]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [8]:
# Create the model
history = autoencoder.fit(X_train, X_train,epochs=10,batch_size=256,shuffle=True,validation_data=(X_test, X_test))

Epoch 1/10
196/196 [==============================] - 5s 27ms/step - loss: 0.6961 - accuracy: 2.8000e-04 - val_loss: 0.6960 - val_accuracy: 4.0000e-04
Epoch 2/10
196/196 [==============================] - 5s 24ms/step - loss: 0.6959 - accuracy: 2.8000e-04 - val_loss: 0.6959 - val_accuracy: 4.0000e-04
Epoch 3/10
196/196 [==============================] - 5s 24ms/step - loss: 0.6958 - accuracy: 2.8000e-04 - val_loss: 0.6958 - val_accuracy: 4.0000e-04
Epoch 4/10
196/196 [==============================] - 5s 25ms/step - loss: 0.6957 - accuracy: 3.4000e-04 - val_loss: 0.6957 - val_accuracy: 4.0000e-04
Epoch 5/10
196/196 [==============================] - 5s 25ms/step - loss: 0.6956 - accuracy: 3.2000e-04 - val_loss: 0.6956 - val_accuracy: 5.0000e-04
Epoch 6/10
196/196 [==============================] - 5s 26ms/step - loss: 0.6955 - accuracy: 3.2000e-04 - val_loss: 0.6955 - val_accuracy: 5.0000e-04
Epoch 7/10
196/196 [==============================] - 5s 26ms/step - loss: 0.6954 - accuracy: 

In [9]:
embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(128, embed_dim,input_length = X_test.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [10]:
# Compile model
model = createmodel()
epochs = 10
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=3072)

Epoch 1/10


In [ ]:
scores = autoencoder.evaluate(X_train,X_train , verbose=1)
print('Test  loss:', scores[0])
print('Test  accuracy:', scores[1])
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(test_data[0,:,:])  
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_test[0,:,:])  
plt.show()

In [ ]:
print(loaded_model.predict_classes(X_test[[1],:]))

In [ ]:
print(test_data[1])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_test[1,:,:])  
plt.show()

In [ ]:
print(loaded_model.predict_classes(X_test[[2],:]))

In [ ]:
print(test_data[2])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_test[2,:,:])  
plt.show()

In [ ]:
print(loaded_model.predict_classes(X_test[[3],:]))

In [ ]:
print(test_data[3])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_test[3,:,:])  
plt.show()

In [ ]:
##ACCURACY
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
## LOSS
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left')
plt.show()